In [1]:
import json
import spacy
nlp = spacy.load("output/")

c:\Users\KSpicer\Anaconda3\envs\victorian_authorship_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
with open('./texts_for_testing/1880_90030.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['plain_text']
spacy_doc = nlp(opinion_of_text)